<a href="https://colab.research.google.com/github/Cranjis-McB/CNN_ARCHITECTURES/blob/main/cnn_architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN ARCHITECTURES

This notebook contains the **Popular CNN Architecures** implemented from the Scratch in **PyTorch** framework.

### **Implemented Architectures**:



*   **ResNet** (Version - 18, 34, 50, 101, 152)
*   **EfficientNet** (Version - b0-b7)
*   **EfficientNet-V2** (Version - S, M, L)





In [1]:
# Import useful Modules.
import torch
from torch import nn
from math import ceil

## Resnet

In [ ]:
# Tier-2 ResBlock class. (Resnet-18,Resnet-34)
class ResBlock_Tier2(nn.Module):
    
    def __init__(self, in_channels, intermediate_channels):
        super(ResBlock_Tier2, self).__init__()
        
        # Layers
        # To make the residual input equal size as output channel.
        
        downsample = False
        self.skip_connection = nn.Sequential() # Default
        
        if intermediate_channels == 2*in_channels:
            self.skip_connection = nn.Sequential(nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=2, bias = False),
                                             nn.BatchNorm2d(intermediate_channels)
                                            )
            downsample = True
            

        
        # Downsampling the output shape.
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=3, stride=2, padding = 1, bias = False)
            
        else:
            self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=3, stride=1, padding = 1, bias = False)
            
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        # Residual to be added later.
        identity = self.skip_connection(x)
        
        #---------------------------
        # Layer-1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #----------------------------
        
        #---------------------------
        # Layer-2
        x = self.conv2(x)
        x = self.bn2(x)
        #print(x.size())
        #print(identity.size())
        x += identity
        x = self.relu(x)
        #----------------------------
        
        return x
    
#---------------------------------------------------------------------------------------------------
'''
  This is Tier-2 Resnet class (Resnet-18, 34)
  It takes input as img_channels:
  for RGB = 3
  for Gray = 1),
  
  num_layers: 
  for Resnet-18 = [2, 2, 2, 2]
  for Resnet-34 = [3, 4, 6, 3]
  
  , and number of classes. 
  for ex : Imagenet = 1000, MNIST = 10 etc
  
  Output : Resnet Model
  
'''
class ResNet_Tier2(nn.Module):
    
    def __init__(self, img_channels, num_layers, num_classes):
        super(ResNet_Tier2, self).__init__()
        
        # Layers 
        # Layer-0 Output shape : 64 X 56 X 56
        self.layer0 = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        # Residual Blocks layers
        self.layer1 = self._make_layer(ResBlock_Tier2, num_layers[0], 64, 64)
        self.layer2 = self._make_layer(ResBlock_Tier2, num_layers[1], 64, 128)
        self.layer3 = self._make_layer(ResBlock_Tier2, num_layers[2], 128, 256)
        self.layer4 = self._make_layer(ResBlock_Tier2, num_layers[3], 256, 512)
        
        # FC Layers
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc  = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock_Tier2, num_residual_blocks, in_channels, intermediate_channels):
        
        layers = []
        only_once = True
        
        for i in range(num_residual_blocks):
            layers.append(ResBlock_Tier2(in_channels, intermediate_channels))
            if only_once:
                in_channels = intermediate_channels
                only_once = False
            
        return nn.Sequential(*layers)

In [ ]:
# Tier-1 ResBlock class. (Resnet-50, Resnet-101, Resnet-152)
class ResBlock_Tier1(nn.Module):
    
    def __init__(self, in_channels, intermediate_channels):
        super(ResBlock_Tier1, self).__init__()
        
        # Layers
        # To make the residual input equal size as output channel.
        
        stride = 1 # Default value
        downsample = False
        
        if intermediate_channels == in_channels/2:
            stride = 2
            downsample = True
            
        self.skip_connection = nn.Sequential(nn.Conv2d(in_channels, intermediate_channels*4, kernel_size=1, stride=stride, bias = False),
                                             nn.BatchNorm2d(intermediate_channels*4)
                                            )
        
        # Downsampling the output shape.
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=2, bias = False)
            
        else:
            self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, bias = False)
            
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv3 = nn.Conv2d(intermediate_channels, intermediate_channels*4, kernel_size=1, stride=1, bias = False)
        self.bn3 = nn.BatchNorm2d(intermediate_channels*4)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        # Residual to be added later.
        identity = self.skip_connection(x)
        
        #---------------------------
        # Layer-1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #----------------------------
        
        #---------------------------
        # Layer-2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        #----------------------------
        
        #---------------------------
        # Layer-3
        x = self.conv3(x)
        x = self.bn3(x)
        #print("Size identity", identity.size())
        #print("Size X", x.size())
        x += identity
        x = self.relu(x)
        #----------------------------
        
        return x
    
#---------------------------------------------------------------------------------------------------
'''
  This is Tier-1 Resnet class (Resnet-50, 101, 152)
  It takes input as img_channels:
  for RGB = 3
  for Gray = 1),
  
  num_layers: 
  for Resnet-50 = [3, 4, 6, 3]
  for Resnet-101 = [3, 4, 23, 3]
  for Resnet-152 = [3, 8, 36, 3]
  
  , and number of classes. 
  for ex : Imagenet = 1000, MNIST = 10 etc
  
  Output : Resnet Model
  
'''
class ResNet_Tier1(nn.Module):
    
    def __init__(self, img_channels, num_layers, num_classes):
        super(ResNet_Tier1, self).__init__()
        
        # Layers 
        # Layer-0 Output shape : 64 X 56 X 56
        self.layer0 = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        # Residual Blocks layers
        self.layer1 = self._make_layer(ResBlock_Tier1, num_layers[0], 64, 64)
        self.layer2 = self._make_layer(ResBlock_Tier1, num_layers[1], 256, 128)
        self.layer3 = self._make_layer(ResBlock_Tier1, num_layers[2], 512, 256)
        self.layer4 = self._make_layer(ResBlock_Tier1, num_layers[3], 1024, 512)
        
        # FC Layers
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc  = nn.Linear(512 * 4, num_classes)
        
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock_Tier1, num_residual_blocks, in_channels, intermediate_channels):
        
        layers = []
        only_once = True
        
        for i in range(num_residual_blocks):
            layers.append(ResBlock_Tier1(in_channels, intermediate_channels))
            if only_once:
                in_channels = intermediate_channels*4
                only_once = False
            
        return nn.Sequential(*layers)

In [ ]:
'''Resnets'''

# Tier-2 Resnets.
def ResNet18(img_channel=3, num_classes=1000):
    return ResNet_Tier2(img_channel, [2, 2, 2, 2], num_classes)


def ResNet34(img_channel=3, num_classes=1000):
    return ResNet_Tier2(img_channel, [3, 4, 6, 3], num_classes)

#--------------------------------------------------------------------

# Tier-1 Resnets.
def ResNet50(img_channel=3, num_classes=1000):
    return ResNet_Tier1(img_channel, [3, 4, 6, 3], num_classes)


def ResNet101(img_channel=3, num_classes=1000):
    return ResNet_Tier1(img_channel, [3, 4, 23, 3], num_classes)


def ResNet152(img_channel=3, num_classes=1000):
    return ResNet_Tier1(img_channel, [3, 8, 36, 3], num_classes)

## EfficientNet

In [ ]:
''' A simple Convolution, Batch Normalization, and Activation Class'''

class ConvBnAct(nn.Module):
    
    def __init__(self, n_in, n_out, kernel_size = 3, stride = 1, 
                 padding = 0, groups = 1, bn = True, act = True,
                 bias = False
                ):
        
        super(ConvBnAct, self).__init__()
        
        self.conv = nn.Conv2d(n_in, n_out, kernel_size = kernel_size,
                              stride = stride, padding = padding,
                              groups = groups, bias = bias
                             )
        self.batch_norm = nn.BatchNorm2d(n_out) if bn else nn.Identity()
        self.activation = nn.SiLU() if act else nn.Identity()
        
    def forward(self, x):
        
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.activation(x)
        
        return x
    
#------------------------------------------------------------------------------

''' Squeeze and Excitation Block '''

class SqueezeExcitation(nn.Module):
    
    def __init__(self, n_in, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        
        
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(n_in, reduced_dim, kernel_size=1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, n_in, kernel_size=1),
            nn.Sigmoid()
        )
       
    def forward(self, x):
        
        y = self.se(x)
        
        return x * y
                                    
#------------------------------------------------------------------------------

''' Stochastic Depth Module'''

class StochasticDepth(nn.Module):
    
    def __init__(self, survival_prob = 0.8):
        super(StochasticDepth, self).__init__()
        
        self.p =  survival_prob
        
    def forward(self, x):
        
        if not self.training:
            return x
        
        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.p
        
        return torch.div(x, self.p) * binary_tensor
        
#-------------------------------------------------------------------------------

''' Residual Bottleneck Block with Expansion Factor = N as defined in Mobilenet-V2 paper
    with Squeeze and Excitation Block and Stochastic Depth. 
'''

class MBConvN(nn.Module):
    
    def __init__(self, n_in, n_out, kernel_size = 3, 
                 stride = 1, expansion_factor = 6,
                 reduction = 4, # Squeeze and Excitation Block
                 survival_prob = 0.8 # Stochastic Depth
                ):
        
        super(MBConvN, self).__init__()
        
        self.skip_connection = (stride == 1 and n_in == n_out) 
        intermediate_channels = int(n_in * expansion_factor)
        padding = (kernel_size - 1)//2
        reduced_dim = int(n_in//reduction)
        
        self.expand = nn.Identity() if (expansion_factor == 1) else ConvBnAct(n_in, intermediate_channels, kernel_size = 1)
        self.depthwise_conv = ConvBnAct(intermediate_channels, intermediate_channels,
                                        kernel_size = kernel_size, stride = stride, 
                                        padding = padding, groups = intermediate_channels
                                       )
        self.se = SqueezeExcitation(intermediate_channels, reduced_dim = reduced_dim)
        self.pointwise_conv = ConvBnAct(intermediate_channels, n_out, 
                                        kernel_size = 1, act = False
                                       )
        self.drop_layers = StochasticDepth(survival_prob = survival_prob)
        
    def forward(self, x):
        
        residual = x
        
        x = self.expand(x)
        x = self.depthwise_conv(x)
        x = self.se(x)
        x = self.pointwise_conv(x)
        
        if self.skip_connection:
            x = self.drop_layers(x)
            x += residual
        
        return x
    

#----------------------------------------------------------------------------------------------

'''Efficient-net Class'''

class EfficientNet(nn.Module):
    
    '''Generic Efficient net class which takes width multiplier, Depth multiplier, and Survival Prob.'''
    
    def __init__(self, width_mult = 1, depth_mult = 1, 
                 dropout_rate = 0.2, num_classes = 1000):
        super(EfficientNet, self).__init__()
        
        last_channel = ceil(1280 * width_mult)
        self.features = self._feature_extractor(width_mult, depth_mult, last_channel)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channel, num_classes)
        )
        
    def forward(self, x):
        
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x.view(x.shape[0], -1))
        
        return x
    
        
    def _feature_extractor(self, width_mult, depth_mult, last_channel):
        
        channels = 4*ceil(int(32*width_mult) / 4)
        layers = [ConvBnAct(3, channels, kernel_size = 3, stride = 2, padding = 1)]
        in_channels = channels
        
        kernels = [3, 3, 5, 3, 5, 5, 3]
        expansions = [1, 6, 6, 6, 6, 6, 6]
        num_channels = [16, 24, 40, 80, 112, 192, 320]
        num_layers = [1, 2, 2, 3, 3, 4, 1]
        strides =[1, 2, 2, 2, 1, 2, 1]
        
        # Scale channels and num_layers according to width and depth multipliers.
        scaled_num_channels = [4*ceil(int(c*width_mult) / 4) for c in num_channels]
        scaled_num_layers = [int(d * depth_mult) for d in num_layers]

        
        for i in range(len(scaled_num_channels)):
             
            layers += [MBConvN(in_channels if repeat==0 else scaled_num_channels[i], 
                               scaled_num_channels[i],
                               kernel_size = kernels[i],
                               stride = strides[i] if repeat==0 else 1, 
                               expansion_factor = expansions[i]
                              )
                       for repeat in range(scaled_num_layers[i])
                      ]
            in_channels = scaled_num_channels[i]
        
        layers.append(ConvBnAct(in_channels, last_channel, kernel_size = 1, stride = 1, padding = 0))
    
        return nn.Sequential(*layers)

In [ ]:
# Compound scaling factors for efficient-net family.
EFFICIENT_NET_CONFIG = {
    # tuple of width multiplier, depth multiplier, resolution, and Survival Prob
    "b0" : (1.0, 1.0, 224, 0.2),
    "b1" : (1.0, 1.1, 240, 0.2),
    "b2" : (1.1, 1.2, 260, 0.3),
    "b3" : (1.2, 1.4, 300, 0.3),
    "b4" : (1.4, 1.8, 380, 0.4),
    "b5" : (1.6, 2.2, 456, 0.4),
    "b6" : (1.8, 2.6, 528, 0.5),
    "b7" : (2.0, 3.1, 600, 0.5)
}

In [ ]:
# Efficient-nets.
def EffNet(version, num_classes=1000):
  width_mult, depth_mult, res, dropout_rate = EFFICIENT_NET_CONFIG[version]
  return EfficientNet(width_mult, depth_mult, dropout_rate, num_classes)

##  EfficientNet-V2

In [2]:
Eff_V2_SETTINGS = {
    # expansion factor, k, stride, n_in, n_out, num_layers, use_fusedMBCONV
    's' : [
        [1, 3, 1, 24, 24, 2, True],
        [4, 3, 2, 24, 48, 4, True],
        [4, 3, 2, 48, 64, 4, True],
        [4, 3, 2, 64, 128, 6, False],
        [6, 3, 1, 128, 160, 9, False],
        [6, 3, 2, 160, 256, 15, False]
    ],
    
    'm' : [
        [1, 3, 1, 24, 24, 3, True],
        [4, 3, 2, 24, 48, 5, True],
        [4, 3, 2, 48, 80, 5, True],
        [4, 3, 2, 80, 160, 7, False],
        [6, 3, 1, 160, 176, 14, False],
        [6, 3, 2, 176, 304, 18, False],
        [6, 3, 1, 304, 512, 5, False]
    ],
    
    'l' : [
        [1, 3, 1, 32, 32, 4, True],
        [4, 3, 2, 32, 64, 7, True],
        [4, 3, 2, 64, 96, 7, True],
        [4, 3, 2, 96, 192, 10, False],
        [6, 3, 1, 192, 224, 19, False],
        [6, 3, 2, 224, 384, 25, False],
        [6, 3, 1, 384, 640, 7, False]
    ]
}

In [3]:
'''A simple Convolution + Batch Normalization + Activation Class'''

class ConvBnAct(nn.Module):
    
    def __init__(
        self,
        n_in, # in_channels
        n_out, # out_channels
        k_size = 3, # Kernel Size
        stride = 1, 
        padding = 0,
        groups = 1, 
        act = True, 
        bn = True, 
        bias = False
    ):
        super(ConvBnAct, self).__init__()
        
        self.conv = nn.Conv2d(n_in, n_out, kernel_size = k_size, stride = stride,
                              padding = padding, groups = groups,bias = bias
                             )
        self.batch_norm = nn.BatchNorm2d(n_out) if bn else nn.Identity()
        self.activation = nn.SiLU() if act else nn.Identity()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.activation(x)
        
        return x
    
#--------------------------------------------------------------------------------------------

'''Squeeze and Excitation Class'''

class SqueezeExcitation(nn.Module):
    
    def __init__(
        self,
        n_in, # In_channels
        reduced_dim
    ):
        super(SqueezeExcitation, self).__init__()
      
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excite = nn.Sequential(nn.Conv2d(n_in, reduced_dim, kernel_size=1),
                                   nn.SiLU(),
                                   nn.Conv2d(reduced_dim, n_in, kernel_size=1),
                                   nn.Sigmoid()
                                   )
        
    def forward(self, x):
        y = self.squeeze(x)
        y = self.excite(y)
            
        return x * y
        
#--------------------------------------------------------------------------------------

''' Stochastic Depth Class'''

class StochasticDepth(nn.Module):
    
    def __init__(
        self,
        survival_prob = 0.8
    ):
        super(StochasticDepth, self).__init__()
        
        self.p =  survival_prob
        
    def forward(self, x):
        
        if not self.training:
            return x
        
        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.p
        
        return torch.div(x, self.p) * binary_tensor
        
#-------------------------------------------------------------------------------

'''MBCONV Class'''

class MBConvN(nn.Module):
    
    def __init__(
        self,
        n_in, # In_channels
        n_out, # out_channels
        k_size = 3, # kernel_size
        stride = 1,
        expansion_factor = 4,
        reduction_factor = 4, # SqueezeExcitation Block
        survival_prob = 0.8 # StochasticDepth Block
    ):
        super(MBConvN, self).__init__()
        reduced_dim = int(n_in//4)
        expanded_dim = int(expansion_factor * n_in)
        padding = (k_size - 1)//2
        
        self.use_residual = (n_in == n_out) and (stride == 1)
        self.expand = nn.Identity() if (expansion_factor == 1) else ConvBnAct(n_in, expanded_dim, k_size = 1)
        self.depthwise_conv = ConvBnAct(expanded_dim, expanded_dim,
                                        k_size, stride = stride,
                                        padding = padding, groups = expanded_dim
                                       )
        self.se = SqueezeExcitation(expanded_dim, reduced_dim)
        self.drop_layers = StochasticDepth(survival_prob)
        self.pointwise_conv = ConvBnAct(expanded_dim, n_out, k_size = 1, act = False)
        
    def forward(self, x):
        
        residual = x.clone()
        x = self.expand(x)
        x = self.depthwise_conv(x)
        x = self.se(x)
        x = self.pointwise_conv(x)
        
        if self.use_residual:
            x = self.drop_layers(x)
            x += residual
            
        return x
    
#--------------------------------------------------------------------------------------

'''Fused-MBCONV Class'''

class FusedMBConvN(nn.Module):
    
    def __init__(
        self,
        n_in, # In_channels
        n_out, # out_channels
        k_size = 3, # kernel_size
        stride = 1,
        expansion_factor = 4,
        reduction_factor = 4, # SqueezeExcitation Block
        survival_prob = 0.8 # StochasticDepth Block
    ):
        super(FusedMBConvN, self).__init__()
        
        reduced_dim = int(n_in//4)
        expanded_dim = int(expansion_factor * n_in)
        padding = (k_size - 1)//2
        
        self.use_residual = (n_in == n_out) and (stride == 1)
        #self.expand = nn.Identity() if (expansion_factor == 1) else ConvBnAct(n_in, expanded_dim, k_size = 1)
        self.conv = ConvBnAct(n_in, expanded_dim,
                              k_size, stride = stride,
                              padding = padding, groups = 1
                             )
        #self.se = SqueezeExcitation(expanded_dim, reduced_dim)
        self.drop_layers = StochasticDepth(survival_prob)
        self.pointwise_conv = nn.Identity() if (expansion_factor == 1) else ConvBnAct(expanded_dim, n_out, k_size = 1, act = False)
        
    def forward(self, x):
        
        residual = x.clone()
        #x = self.conv(x)
        x = self.conv(x)
        #x = self.se(x)
        x = self.pointwise_conv(x)
        
        if self.use_residual:
            x = self.drop_layers(x)
            x += residual
            
        return x
    
#-----------------------------------------------------------------------------------------------

class EfficientNetV2(nn.Module):
    
    def __init__(
    self,
    version = 's',
    dropout_rate = 0.2,
    num_classes = 1000
    ):
        super(EfficientNetV2, self).__init__()
        last_channel = 1280
        self.features = self._feature_extractor(version, last_channel)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Dropout(dropout_rate, inplace = True),
            nn.Linear(last_channel, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        
        return x
        
    def _feature_extractor(self, version, last_channel):
        
        # Extract the Config
        config = Eff_V2_SETTINGS[version]
        
        layers = []
        layers.append(ConvBnAct(3, config[0][3], k_size = 3, stride = 2, padding = 1))
        #in_channel = config[0][3]
        
        for (expansion_factor, k, stride, n_in, n_out, num_layers, use_fused) in config:
            
            if use_fused:
                layers += [FusedMBConvN(n_in if repeat==0 else n_out, 
                                        n_out,
                                        k_size=k,
                                        stride = stride if repeat==0 else 1,
                                        expansion_factor=expansion_factor
                                       ) for repeat in range(num_layers)
                          ]
            else:
                
                layers += [MBConvN(n_in if repeat==0 else n_out, 
                                   n_out,
                                   k_size=k,
                                   stride = stride if repeat==0 else 1,
                                   expansion_factor=expansion_factor
                                   ) for repeat in range(num_layers)
                      ]
                
        layers.append(ConvBnAct(config[-1][4], last_channel, k_size = 1))   
            
        return nn.Sequential(*layers)

In [4]:
# Efficient-nets.
def EffNetV2(version, num_classes=1000):
  return EfficientNetV2(version = version, num_classes = num_classes)

--------------------------------------------------------------------------